In [ ]:
## success data
import os
import shutil
import re

# 원본 경로와 저장 경로
raw_root = "/AILAB-summer-school-2025/sucess_raw"
save_root = "./validation_data/success_img"
os.makedirs(save_root, exist_ok=True)

# 에피소드 인덱스를 새로 지정
episode_counter = 0

# 모든 simulation_traj 폴더 순회 (정렬된 순서대로)
for folder in sorted(os.listdir(raw_root)):
    if not folder.startswith("simulation_traj_"):
        continue

    full_path = os.path.join(raw_root, folder)
    if not os.path.isdir(full_path):
        continue

    # steps 정보 파싱
    match = re.search(r"_steps(\d+)_len(\d+)_success", folder)
    if not match:
        continue
    steps = match.group(1)

    # 새로운 에피소드 이름 생성: 순서대로 카운트
    episode_key = f"success_episode{episode_counter}_steps{steps}"
    episode_counter += 1  # 다음 에피소드 번호 증가

    # 저장 경로 생성
    episode_dir = os.path.join(save_root, episode_key)
    os.makedirs(episode_dir, exist_ok=True)

    # 이미지 파일 복사 및 view 폴더 분류
    for img_file in os.listdir(full_path):
        if not img_file.endswith(".png"):
            continue
        view_match = re.match(r"(\w+_view)_(\d+)\.png", img_file)
        if not view_match:
            continue
        view_type, idx = view_match.groups()

        # view별 하위 폴더 생성
        view_folder = os.path.join(episode_dir, view_type)
        os.makedirs(view_folder, exist_ok=True)

        # 이미지 복사
        src = os.path.join(full_path, img_file)
        dst = os.path.join(view_folder, img_file)
        shutil.copy2(src, dst)

print(f"[INFO] 총 {episode_counter}개의 success_episode가 '{save_root}'에 정리되었습니다.")

In [ ]:
import numpy as np
from PIL import Image
import os
import re

# === 경로 설정 ===
traj_dir = "/AILAB-summer-school-2025/sucess_raw/simulation_traj_0_20250806_094019_len455_success"
state_file = os.path.join(traj_dir, "states_440.npz")
output_file = "./success_dataset/success_episode0_steps316.npz"  # 저장 경로
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# === 1. 상태 정보 로드 ===
data = np.load(state_file)
save_dict = {
    "observation/joint_obj_gripper": data["joint_obj_gripper"],
    "observation/EE_pose": data["EE_pose"],
    "observation/torque": data["torque"]
}

# === 2. 이미지 파일 로드 및 저장용 dict 생성 ===
for fname in sorted(os.listdir(traj_dir)):
    if not fname.endswith(".png"):
        continue

    match = re.match(r"(front|top|wrist)_view_(\d+)\.png", fname)
    if not match:
        continue

    view, idx = match.groups()
    key = f"img/{view}/{view}_view_{idx}"

    # 이미지 로드 및 numpy 변환
    img_path = os.path.join(traj_dir, fname)
    img = Image.open(img_path).convert("RGB")
    save_dict[key] = np.array(img)

# === 3. 저장 ===
np.savez_compressed(output_file, **save_dict)
print(f"[INFO] Saved combined npz to: {output_file}")